In [1]:
import numpy as np
import pandas as pd
from hebo.design_space.design_space import DesignSpace
from hebo.optimizers.hebo import HEBO

In [2]:
dimensions = [2, 2, 3, 4, 4, 5, 6, 8]

Xs = []
Ys = []
# Read the first set of inputs
for i, dim in enumerate(dimensions, start=1):
    Xs.append(np.load(f"initial_data/function_{i}/initial_inputs.npy"))
    Y = np.load(f"initial_data/function_{i}/initial_outputs.npy")
    if i == 4:
        Y = np.abs(Y)
    Ys.append(Y)

# Append the second set
for i, dim in enumerate(dimensions, start=1):
    Xs.append(np.load(f"initial_data2/function_{i}/initial_inputs.npy"))
    Y = np.load(f"initial_data2/function_{i}/initial_outputs.npy")
    if i == 4:
        Y = np.abs(Y)
    Ys.append(Y)

# Stack the observations
df = pd.read_csv('feedback_data/605_data.csv')
df = df.drop(columns=['Unnamed: 0', 'timestamp', 'student_id'], axis=1)

for index, col in enumerate(df.columns):
    if 'output' in col:
        y1 = np.log(df[col]) if col == 'f2_output' else df[col]
        y_array = np.array(y1, dtype=float)
        Ys[index - 8] = np.hstack((y_array, Ys[index - 8]))
        continue
    series = []
    for i in range(len(df[col])):
        series.append(df[col][i].replace("[", "").replace("]", "").split())
    series_array = np.array(series, dtype=float)
    Xs[index] = np.vstack((series_array, Xs[index]))

# Use `observe` and `suggest` from HEBO. See  https://github.com/huawei-noah/HEBO/tree/master/HEBO
def run_optimizer(dim, X, Y):
    space = DesignSpace().parse([{'name': f'x{i+1}', 'type': 'num', 'lb': 0, 'ub': 1} for i in range(dim)])
    opt = HEBO(space)
    initial_data = pd.DataFrame(X, columns=[f'x{i+1}' for i in range(dim)])
    initial_data['y'] = Y
    opt.observe(initial_data, Y)

    rec = opt.suggest(n_suggestions=1)

    print('-'.join([f'{value:.6f}' for value in rec.iloc[0].values]))

for i, dim in enumerate(dimensions):
    run_optimizer(dim, Xs[i], Ys[i])

0.999048-0.997971
0.067282-0.944421
0.475366-0.997094-0.999636
0.873948-0.890700-0.780007-0.532901
0.266195-0.755522-0.039836-0.001264
0.000167-0.687637-0.014225-0.000070-0.974107
0.984071-0.002540-0.074480-0.996013-0.442492-0.002424
0.996463-0.818706-0.974459-0.650443-0.729112-0.631887-0.949804-0.221673
